In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import joblib
from scipy.stats import uniform
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


### evaluation metric = f1 score

#### tfidf via custom functionBySven

In [2]:
!pip install gensim

%run ../../../../../functions/vectorize_functions.py


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Load training data
df_train = pd.read_csv("../../../../../../data/mixed_dataset/train_cleaned.csv")
df_train = df_train.dropna(subset=['tweet_cleaned'])

# TF-IDF Vectorization on training data + saving it 
X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

with open("svm_tfidf_vectorizer_opt.joblib", "wb") as file:
    joblib.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['sigmoid'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../../../../../../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_sigmoid_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_sigmoid_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_sigmoid_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_sigmoid_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_sigmoid_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.9152093534525831, 'recall': 0.7147539704218208, 'f1-score': 0.8026555071098356, 'support': 25627}, '1': {'precision': 0.412945711532284, 'recall': 0.7518643076473168, 'f1-score': 0.5330983360116117, 'support': 6839}, 'accuracy': 0.7225713053656132, 'macro avg': {'precision': 0.6640775324924335, 'recall': 0.7333091390345687, 'f1-score': 0.6678769215607236, 'support': 32466}, 'weighted avg': {'precision': 0.809406943328363, 'recall': 0.7225713053656132, 'f1-score': 0.7458729809858673, 'support': 32466}}


In [ ]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_linear_vectorizer_opt.joblib", "wb") as file:
#     joblib.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['linear'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieving best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../../../../../../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_linear_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_linear_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_linear_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_linear_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_linear_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.9223578689278235, 'recall': 0.704608420806181, 'f1-score': 0.798911600743297, 'support': 25627}, '1': {'precision': 0.4126774769183024, 'recall': 0.7777452843983038, 'f1-score': 0.5392335766423357, 'support': 6839}, 'accuracy': 0.7200147846978378, 'macro avg': {'precision': 0.667517672923063, 'recall': 0.7411768526022424, 'f1-score': 0.6690725886928164, 'support': 32466}, 'weighted avg': {'precision': 0.8149931119219368, 'recall': 0.7200147846978378, 'f1-score': 0.7442101282235387, 'support': 32466}}


In [5]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_poly_vectorizer_opt.joblib", "wb") as file:
#     joblib.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['poly'],
    'degree': [3, 4, 5],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../../../../../../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_poly_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_poly_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_poly_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_poly_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_poly_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 25627}, '1': {'precision': 0.21065114273393704, 'recall': 1.0, 'f1-score': 0.34799643811219944, 'support': 6839}, 'accuracy': 0.21065114273393704, 'macro avg': {'precision': 0.10532557136696852, 'recall': 0.5, 'f1-score': 0.17399821905609972, 'support': 32466}, 'weighted avg': {'precision': 0.044373903935113514, 'recall': 0.21065114273393704, 'f1-score': 0.07330584735567461, 'support': 32466}}


c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\fried\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [6]:
# # Load training data
# df_train = pd.read_csv("../data/mixed_dataset/train_cleaned.csv")
# df_train = df_train.dropna(subset=['tweet_cleaned'])

# # TF-IDF Vectorization on training data + saving it 
# X_train, X_test, y_train, y_test, tfidf_vectorizer = vectorize_tfidf(df=df_train, text_column="tweet_cleaned", label_column="label")

# with open("svm_tfidf_rbf_vectorizer_opt.joblib", "wb") as file:
#     joblib.dump(tfidf_vectorizer, file)

# Hyperparameter search for SVC-model
param_dist = {
    'kernel': ['rbf'],
    'C': uniform(0.01, 1),
    'gamma': uniform(0.01, 0.1),
    'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}]
}

random_search = RandomizedSearchCV(
    SVC(),
    param_distributions=param_dist,
    n_iter=10, 
    scoring='f1',
    cv=2, 
    verbose=1,  
    n_jobs=-1,  
    random_state=42,
    refit=True 
)

random_search.fit(X_train, y_train)

# Retrieing best model found
best_model = random_search.best_estimator_

# Load test data
df_test = pd.read_csv("../../../../../../data/mixed_dataset/test_cleaned.csv")
df_test = df_test.dropna(subset=['tweet_cleaned'])

# Use vectorizer to transform test data
X_test = tfidf_vectorizer.transform(df_test['tweet_cleaned'])
y_test = df_test["label"]

# Evaluate model on test set
y_test_pred = best_model.predict(X_test)
test_report = classification_report(y_test, y_test_pred, output_dict=True)
cm = confusion_matrix(y_test, y_test_pred)

#--------------------------------------
# Calculate additional metrics and add them to classification report df
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, average='weighted')
recall = recall_score(y_test, y_test_pred, average='weighted')
f1 = f1_score(y_test, y_test_pred, average='weighted')

classification_report_df = pd.DataFrame(test_report).transpose()

classification_report_df['accuracy'] = accuracy
classification_report_df['precision'] = precision
classification_report_df['recall'] = recall
classification_report_df['f1_score'] = f1
#----------------------

print("Test Classification Report:")
print(test_report)

# Save best model and hyperparameter search results
with open("svm_tfidf_rbf_best_model_opt.joblib", "wb") as file:
    joblib.dump(best_model, file)

results_df = pd.DataFrame(random_search.cv_results_)
results_df.to_csv("svm_tfidf_rbf_results_opt.csv", index=False)

# Save configuration of best model
best_model_config = best_model.get_params()
best_model_config_df = pd.DataFrame([best_model_config])
best_model_config_df.to_csv("svm_tfidf_rbf_best_model_config_opt.csv", index=False)

# Save confusion matrix
unique_labels = sorted(y_test.unique())
cm_df = pd.DataFrame(cm, 
                     index=[f"Actual_{label}" for label in unique_labels],
                     columns=[f"Predicted_{label}" for label in unique_labels])
cm_df.to_csv("svm_tfidf_rbf_confusion_matrix_opt.csv", index=True)

# Save classification report
classification_report_df = pd.DataFrame(test_report).transpose()
classification_report_df.to_csv("svm_tfidf_rbf_classification_report_opt.csv", index=True)


Fitting 2 folds for each of 10 candidates, totalling 20 fits
Test Classification Report:
{'0': {'precision': 0.9191745485812554, 'recall': 0.7091348967885434, 'f1-score': 0.8006079562976344, 'support': 25627}, '1': {'precision': 0.41283970066955494, 'recall': 0.7663401082029536, 'f1-score': 0.5366028463192383, 'support': 6839}, 'accuracy': 0.7211852399433253, 'macro avg': {'precision': 0.6660071246254051, 'recall': 0.7377375024957484, 'f1-score': 0.6686054013084364, 'support': 32466}, 'weighted avg': {'precision': 0.8125145342626415, 'recall': 0.7211852399433253, 'f1-score': 0.7449949781930865, 'support': 32466}}
